# Importo las librerías necesarias

In [23]:
import pandas as pd #pandas es la librería estándard para el manejo de dataframes
import plotly.graph_objects as go #Plotly es una libería gráfica interactiva
import numpy as np #numpy es una librería para manipular arrays (no se asusten)

Links importantes:
- Librerías gráficas en python
  - https://matplotlib.org/stable/gallery/index.html
  - https://seaborn.pydata.org/examples/index.html
  - https://plotly.com/python/
- Información adicional de plotly
  - https://plotly.com/python/shapes/
  - https://plotly.com/python/templates/
  - https://plotly.com/python/splom/
- Inteligencia artificial
  - [phind.com](https://www.phind.com/) (recomendada)
  - [chatGPT](https://chat.openai.com/)
- Proyectos finales Python (análisis de datos)
  - [dashboards con plotly](https://plotly.com/examples/)
  - https://www.indec.gob.ar/ftp/ica_digital/ica_d_11_23EB819483A2/
  - [Manipular excels](https://xlsxwriter.readthedocs.io/)
  - [Fast API](https://fastapi.tiangolo.com/)
  - [Web Scraping](https://selenium-python.readthedocs.io/getting-started.html#simple-usage)

# Importo el dataframe y lo "limpio"

In [24]:
'''Leo la DataFrame desde la publicación, donde el nombre de las columnas están en la segunda fila.
La columna "Años" debería ser del tipo string, dado a que existen valores que terminan con *,
para hacer referencia a los valores estimados'''

df = pd.read_excel("https://www.indec.gob.ar/ftp/cuadros/economia/balan_1910_2022.xls",
                  header=2,
                  dtype=dict(Años = str))
df.columns = [columna.strip() for columna in df.columns] #Limpio los nombres de las columnas (habían algunas con espacios raros)
df = (df.dropna(subset="Exportación") #Elimino los valores sin valor de la columna "Exportación"
      .reset_index(drop = True) # Reseteo el índice
      .dropna(axis=1) #Elimino las columnas que contengan valores vacíos
      .rename({"Años" : "Año"},axis=1) #Renombro la columna "Años"
      )
df.Año = df.Año.str.replace("*","", regex=False) #"Elimino las observaciones con asteriscos"

#Expreso en millones de US$
df["Exportación"] = df["Exportación"] / 1_000
df["Importación"] = df["Importación"] / 1_000
df["Saldo"] = df["Exportación"] - df["Importación"]
df

,Año,Exportación,Importación,Saldo
0,1910,373.122363,363.797468,9.324895
1,1911,328.270042,388.396624,-60.126582
2,1912,481.097046,428.523207,52.573840
3,1913,497.848101,475.864979,21.983122
4,1914,388.220339,310.593220,77.627119
...,...,...,...,...
108,2018,61781.528000,65482.815000,-3701.287000
109,2019,65115.325000,49125.031000,15990.294000
110,2020,54883.822346,42355.636878,12528.185468
111,2021,77934.314000,63183.753000,14750.561000


# Análisis gráfico de los resultados

## Primeros pasos

In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df["Año"], y = df["Exportación"], mode="lines + markers",))
fig.update_layout(template = None, title_text = "Evolución histórica de las exportaciones argentinas<br><sup>En millones de US$")
fig.update_yaxes(title_text = "Millones de US$")
fig.update_xaxes(title_text = "Fecha",
                #  range = [1994, 2023]
                 )
fig.show()

In [26]:
temp = df.query('Año >= "1990"')
fig = go.Figure()
fig.add_trace(go.Scatter(x = temp["Año"], y = temp["Exportación"], mode="lines + markers"))
fig.update_layout(template = None, title_text = "Evolución histórica de las exportaciones argentinas<br><sup>En millones de US$",
                  height = 600, width = 800,
                  separators = ",."
                  ) # Para que los miles se separen por "." y los decimales en ","
fig.update_yaxes(title_text = "Millones de US$",
                 tickformat = ",") # Para que se separe en miles

fig.update_xaxes(title_text = "Fecha", 
                 type = "category", 
                 nticks = 10, #Para evitar que se vea como un número continuo
                 ) 

In [27]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.Año, y = df.Saldo))
fig.update_layout(
    template = None,
    title_text = "Evolución de la balanza comercial<br><sup>En millones de US$",
                  font_family = "georgia")
fig.update_yaxes(title_text = "Millones de US$")
fig.update_xaxes(title_text = "Año")

In [28]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.Año, y = df.Exportación, name = "Exportaciones"))
fig.add_trace(go.Bar(x=df.Año, y = df.Importación, name = "Importaciones"))
fig.update_layout(template = None, title_text = "Evolución de la balanza comercial<br><sup>En millones de US$",
                  font_family = "georgia", 
                  barmode = "stack"
                  )
fig.update_yaxes(title_text = "Millones de US$")
fig.update_xaxes(title_text = "Año")

In [29]:
temp = df.query("Año >= '1990'")
fig = go.Figure()
fig.add_trace(go.Scatter(x = temp["Año"], y = temp["Exportación"], 
                         name = "Exportación", mode = "lines", marker_color = "red",
                         hovertemplate="%{y:,.0f} millones de US$"))
fig.add_trace(go.Scatter(x = temp["Año"], y = temp["Importación"], name = "Importación", mode = "lines+markers", marker_symbol = 3))
fig.add_trace(go.Bar(x = temp["Año"], y = temp["Saldo"], name = "Saldo"))
fig.update_layout(template = None, title_text = "Evolución del Intercambio Comercial Argentino<br><sup>En millones de US$",
                   font_family = "georgia",
                   height = 600, width = 1200)
fig.update_yaxes(title_text = "Millones de US$")
fig.update_xaxes(title_text = "Año")

In [30]:
temp = df.query("Año >= '1990'")
fig = go.Figure()
fig.add_trace(go.Scatter(x = temp["Año"], y = temp["Exportación"], 
                         name = "Exportación", mode = "lines", marker_color = "red", 
                         hovertemplate="%{y:,.1f}"))
fig.add_trace(go.Scatter(x = temp["Año"], y = temp["Importación"], name = "Importación", mode = "lines+markers", marker_symbol = 0))
fig.add_trace(go.Bar(x = temp["Año"], y = temp["Saldo"], name = "Saldo"))
fig.update_layout(template = None, title_text = "Evolución del Intercambio Comercial Argentino<br><sup>En millones de US$",
                   font_family = "georgia", hovermode = "x unified", #seteo el hover en un estilo especial y lindo
                   height = 600, width = 1200,
                   separators = ",.")
fig.update_yaxes(title_text = "Millones de US$",
                 tickformat = ",") #!!! importante
fig.update_xaxes(title_text = "Año", showgrid=True)

In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = temp["Importación"], y = temp["Exportación"], mode = "markers",
                        marker=dict(
                            size = abs(temp["Saldo"] / temp["Saldo"].mean() * 10),
                            color = temp["Saldo"],),
                        customdata=np.transpose([temp["Año"], temp["Saldo"]]),
                        hovertemplate="<b>%{customdata[0]}</b><extra></extra><br>Exportaciones: %{y:,.1f}<br>Importaciones: %{x:,.1f}<br>Saldo: %{customdata[1]:,.1f}"
                        ))

min_val = min(temp["Exportación"].min(), temp["Importación"].min())
max_val = max(temp["Exportación"].max(), temp["Importación"].max())

fig.add_shape(
    type='line',
    x0=min_val,
    y0=min_val,
    x1=max_val,
    y1=max_val,
    line=dict(color='red', width=2, dash = "dash")
)

fig.update_layout(
    template = None,
    separators = ",.",)
fig.update_yaxes(tickformat = ",", title_text = "Exportaciones")
fig.update_xaxes(tickformat = ",", title_text = "Importaciones")

In [32]:
import numpy as np

templates = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

for template in templates:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = temp["Importación"], y = temp["Exportación"], mode = "markers",
                            marker=dict(
                                size = abs(temp["Saldo"] / temp["Saldo"].mean() * 10),
                                color = temp["Saldo"],),
                            customdata=np.transpose([temp["Año"], temp["Saldo"]]),
                            hovertemplate="<b>%{customdata[0]}</b><extra></extra><br>Exportaciones: %{y:,.1f}<br>Importaciones: %{x:,.1f}<br>Saldo: %{customdata[1]:,.1f}"
                            ))

    min_val = min(temp["Exportación"].min(), temp["Importación"].min())
    max_val = max(temp["Exportación"].max(), temp["Importación"].max())

    fig.add_shape(
        type='line',
        x0=min_val,
        y0=min_val,
        x1=max_val,
        y1=max_val,
        line=dict(color='red', width=2, dash = "dash")
    )

    fig.update_layout(
        template = template, 
        title_text = f"Template: {template}",
        separators = ",.",)
    fig.update_yaxes(tickformat = ",", title_text = "Exportaciones")
    fig.update_xaxes(tickformat = ",", title_text = "Importaciones")
    fig.add_annotation(text = "Fuente: INDEC", x=0,y=-.15, showarrow=False, xref="paper",yref="paper")
    fig.show()

Recomendación para usar en pdf: exportar en formato html, luego "imprimirlo" para guardarlo en pdf donde se quiera. 

In [33]:
fig.write_html("grafico muy importante.html")

In [34]:
df = pd.read_excel("./data/indice de produccion industrial minero.xlsx", sheet_name=1)
df["Fecha"] =  df["Mes"].apply(lambda x: x[:3].lower()) + "-" + df["Año"].astype(str).apply(lambda x: x[2:])
df

,Año,Mes,Serie original,Serie desestacionalizada,Serie tendencia-ciclo,Fecha
0,2017,Enero,97.108796,98.174007,98.279502,ene-17
1,2017,Febrero,87.129209,96.282681,98.553573,feb-17
2,2017,Marzo,99.588022,97.880259,98.954391,mar-17
3,2017,Abril,95.397731,100.191934,99.461312,abr-17
4,2017,Mayo,102.335582,101.327478,100.111238,may-17
...,...,...,...,...,...,...
76,2023,Mayo,122.587025,121.935725,122.020481,may-23
77,2023,Junio,120.230036,122.625611,122.542869,jun-23
78,2023,Julio,123.856698,123.221102,122.938399,jul-23
79,2023,Agosto,128.556573,123.723881,123.173371,ago-23


In [35]:
titulo = "<b>IPI minero Nivel general. Serie original, desestacionalizada y tendencia-ciclo, base 2016=100, en números<br>índice. Enero 2017-septiembre 2023"
fig = go.Figure()
fig.add_trace(go.Scatter(x = df["Fecha"], y = df["Serie original"],name = "Serie original", mode = "lines + markers",
                         line_color = "black",hovertemplate="%{y:,.1f}"))
fig.add_trace(go.Scatter(x = df["Fecha"],y = df["Serie desestacionalizada"], name = "Serie desestacionalizada", line_dash = "dash",
                         line_width = 3, hovertemplate="%{y:,.1f}"))
fig.add_trace(go.Scatter(x = df["Fecha"], y = df["Serie tendencia-ciclo"], name = "Serie tendencia-ciclo", line_width = 3,
                        #  hovertemplate="%{y:,.1f}"
                         ))
fig.update_layout(template = None, hovermode = "x unified", title_text = titulo,
                  separators = ",.",
                  legend = dict(
                      orientation = "h",
                      y = -.12,
                      x = .3
                  )
                  )
fig.update_xaxes(nticks = 20)
fig.update_yaxes(tickformat = ",")
fig.add_vrect(
    x0="mar-20", x1="jul-20",
    fillcolor="red", opacity=0.5,
    layer="above", line_width=0,)
fig.add_annotation(text = "Pandemia", x="may-20", y = 120, showarrow=False, textangle=90, 
                   font={
                       "family":"georgia",
                       "size":15
})